In [1]:
%load_ext autoreload
%load_ext autotime

time: 186 µs (started: 2024-10-07 23:39:00 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 571 µs (started: 2024-10-07 23:39:00 -07:00)


In [3]:
import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
print('pandas', pd.__version__)
print('dask', dask.__version__)
print('numpy', np.__version__)

pandas 2.2.3
dask 2023.5.1
numpy 1.24.3
time: 1.73 s (started: 2024-10-07 23:39:00 -07:00)


In [4]:
analyzer = "dftracer"
checkpoint = False
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
trace_path = "/p/lustre3/iopp/dftracer-traces/v1.0.5-develop/corona/cm1_1_48_20240926/cm1_1_48_20240926-COMPACT/*.pfw.gz"

time: 985 µs (started: 2024-10-07 23:39:02 -07:00)


In [5]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-bert-v100-node-4-v1"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/bert-v100-node-4-v1"
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/bert/v100/node-4/v1/RAW/*.pfw.gz"


time: 675 µs (started: 2024-10-07 23:39:02 -07:00)


In [6]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 4.88 s (started: 2024-10-07 23:39:02 -07:00)


In [7]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    threshold=cfg.threshold,
    view_types=cfg.view_types,
)

time: 21.6 s (started: 2024-10-07 23:39:07 -07:00)


In [8]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          1097.44 seconds                                                                               │
│  I/O Time         3.41 seconds                                                                                  │
│                   ├── Read - 1.00 seconds (29.30%)                                                              │
│                   ├── Write - 0.01 seconds (0.25%)                                                              │
│                   └── Metadata - 2.49 seconds (72.85%)                                                          │
│  I/O Operations   2,175,579 ops                                                                                 │
│                   ├── Read - 72,292 ops (3.32%)                                                                 │
│                   ├── Write - 2,029 ops (0.09%)                                                                 │
│                   └── Metadata - 2,092,426 ops (96.18%)                                                         │
│  I/O Size         3.94 GiB                                                                                      │
│                   ├── Read - 3.94 GiB (99.99%)                                                                  │
│                   └── Write - 248.63 kiB (0.01%)                                                                │
│  Read Requests    4 kiB-256 kiB - 72,164 ops                                                                    │
│                   ├── <4 kiB - 43,632 ops (60.46%)                                                              │
│                   ├── 4-16 kiB - 4,992 ops (6.92%)                                                              │
│                   ├── 16-64 kiB - 7,872 ops (10.91%)                                                            │
│                   └── 64-256 kiB - 15,668 ops (21.71%)                                                          │
│  Write Requests   4 kiB-4 kiB - 2,029 ops                                                                       │
│                   └── <4 kiB - 2,029 ops (100.00%)                                                              │
│  Nodes            4 nodes                                                                                       │
│                   ├── c51182251654b704 - 3.41 s (100.00%) - 0.98 GiB/10.55 kiB R/W (24.99/0.00%) - 543,587 ops  │
│                   │   (24.99%)                                                                                  │
│                   ├── b14bffbb1056730d - 3.37 s (98.66%) - 0.98 GiB/10.55 kiB R/W (24.97/0.00%) - 543,593 ops   │
│                   │   (24.99%)                                                                                  │
│                   ├── 0b2e877f7c065ff0 - 3.36 s (98.32%) - 0.99 GiB/10.55 kiB R/W (25.02/0.00%) - 543,599 ops   │
│                   │   (24.99%)                                                                                  │
│                   └── aeed2ff63b9a3441 - 3.33 s (97.54%) - 0.99 GiB/216.99 kiB R/W (25.02/0.01%) - 544,800 ops  │
│                       (25.04%)                                                                                  │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  709 processes                                                                                 │
│  Files            5,065 files                                                                                   │
│                   ├── Shared: 4,775 files (94.27%)                                                              │
│                   └── FPP: 290 files (5.73%)          

time: 2.59 s (started: 2024-10-07 23:39:28 -07:00)
